<a href="https://colab.research.google.com/github/lucavgn/AML_Project5/blob/main/Centr_CIFAR_100_80split20_AdamW_SGDM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CIFAR 100 - CENTRALIZED TRAINING

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import random_split

# Training settings
epochs = 150  # Adjust epochs based on computational resources
batch_size = 128
momentum = 0.9
weight_decay = 1e-2
# Define LeNet-5 architecture
class LeNet5(nn.Module):
      def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 5)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(64, 64, 5)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 5 * 5, 384)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(384, 192)
        self.relu4 = nn.ReLU()
        self.fc3 = nn.Linear(192, 100)

      def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = x.view(-1, 64 * 5 * 5)
        x = self.relu3(self.fc1(x))
        x = self.relu4(self.fc2(x))
        x = self.fc3(x)
        return x

# Load and split CIFAR-100 dataset
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

torch.manual_seed(42) #Set the seed for reproducibility of random split
torch.cuda.manual_seed_all(42) #set the seed for reproducibility of random operation of the GPU

dataset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
trainset, valset = random_split(dataset, [train_size, val_size]) #Split the dataset

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, num_workers=2)

testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

# Device setup
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Initialize models
net_sgdm = LeNet5().to(device)
net_adamw = LeNet5().to(device)
# Loss function
criterion = nn.CrossEntropyLoss()
# Optimizers and schedulers
sgdm_optimizer = optim.SGD(net_sgdm.parameters(), lr=0.01, momentum=momentum, weight_decay=weight_decay)
adamw_optimizer = optim.AdamW(net_adamw.parameters(), lr=1e-4, weight_decay=weight_decay)
scheduler_sgdm = optim.lr_scheduler.CosineAnnealingLR(sgdm_optimizer, T_max=150)
scheduler_adamw = optim.lr_scheduler.CosineAnnealingLR(adamw_optimizer, T_max=150)

# Training function
def train_model(optimizer, scheduler, model, criterion, trainloader, valloader, device, epochs):
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []
    for epoch in range(epochs):
        model.train()
        running_loss, correct_train, total_train = 0.0, 0, 0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total_train += labels.size(0)
            correct_train += predicted.eq(labels).sum().item()
        train_loss = running_loss / len(trainloader)
        train_accuracy = 100. * correct_train / total_train
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)
        model.eval()
        correct_val, total_val, val_loss = 0, 0, 0.0
        with torch.no_grad():
            for inputs, labels in valloader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                total_val += labels.size(0)
                correct_val += predicted.eq(labels).sum().item()
        val_loss /= len(valloader)
        val_losses.append(val_loss)
        val_accuracy = 100. * correct_val / total_val
        val_accuracies.append(val_accuracy)
        scheduler.step()
        print(f'Epoch {epoch+1}/{epochs}, Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, Val Acc: {val_accuracy:.2f}%')
    return train_losses, val_losses, train_accuracies, val_accuracies

# Train models
print("--- Train with AdamW ---")
adamw_train_loss, adamw_val_loss, adamw_train_acc, adamw_val_acc = train_model(adamw_optimizer, scheduler_adamw, net_adamw, criterion, trainloader, valloader, device, epochs)
# Save model
torch.save(net_adamw.state_dict(), 'net_adamw.pth')
# Plot metrics
plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 1)
plt.plot(adamw_train_loss, label='AdamW Train Loss')
plt.title('Training Loss')
plt.legend()

plt.subplot(2, 2, 2)
plt.plot(adamw_val_loss, label='AdamW Validation Loss')
plt.title('Validation Loss')
plt.legend()
plt.subplot(2, 2, 3)
plt.plot(adamw_train_acc, label='AdamW Train Accuracy')
plt.title('Training Accuracy')
plt.legend()
plt.subplot(2, 2, 4)
plt.plot(adamw_val_acc, label='AdamW Validation Accuracy')
plt.title('Validation Accuracy')
plt.legend()
plt.tight_layout()
plt.savefig('metrics.png')
plt.show()


Files already downloaded and verified
Files already downloaded and verified
--- Train with AdamW ---
Epoch 1/150, Loss: 4.3175, Train Acc: 4.80%, Val Acc: 7.60%
Epoch 2/150, Loss: 4.0203, Train Acc: 8.89%, Val Acc: 10.27%
Epoch 3/150, Loss: 3.8836, Train Acc: 11.12%, Val Acc: 11.67%
Epoch 4/150, Loss: 3.7719, Train Acc: 12.75%, Val Acc: 13.92%
Epoch 5/150, Loss: 3.6805, Train Acc: 14.46%, Val Acc: 14.56%
Epoch 6/150, Loss: 3.5922, Train Acc: 15.88%, Val Acc: 16.43%
Epoch 7/150, Loss: 3.5142, Train Acc: 16.95%, Val Acc: 17.83%
Epoch 8/150, Loss: 3.4519, Train Acc: 18.17%, Val Acc: 17.88%
Epoch 9/150, Loss: 3.3954, Train Acc: 19.00%, Val Acc: 19.56%
Epoch 10/150, Loss: 3.3522, Train Acc: 19.76%, Val Acc: 19.51%
Epoch 11/150, Loss: 3.3065, Train Acc: 20.41%, Val Acc: 20.89%
Epoch 12/150, Loss: 3.2720, Train Acc: 21.16%, Val Acc: 21.04%
Epoch 13/150, Loss: 3.2396, Train Acc: 21.79%, Val Acc: 21.82%
Epoch 14/150, Loss: 3.2089, Train Acc: 22.30%, Val Acc: 21.83%
Epoch 15/150, Loss: 3.1691, T